# Tutorial 26: Data files and I/O

## PHYS 2600, Spring 2019

In [1]:
# Import cell
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import csv


## T26.1 - Basic I/O and the CSV format

Let's start with some basic file parsing.

### Part A

To begin with, I've provided a comma-separated value file called `example_data_tut25.csv`.  CSV files are tabular data, with the rows separated by newlines and the columns within each row separated by commas.

Since a CSV file is human-readable, a good place to start is to __open the file and look at its contents.__  You should see how many rows and columns to expect.  You'll also notice that this particular file contains a __header__: the very first line of the file doesn't contain data, but instead a set of strings that describe the data columns below.  (Data which exists to tell us generic information about other data like this is usually called __metadata__.)

The cell below contains an example line of data from this CSV file, formatted as a string.  To get the numbers out as a list, you should carry out the following steps:

1. Use the `.strip()` string method to get rid of the newline character `\n`.
2. Use the `.split()` string method to divide the string into smaller strings.
3. Convert each string in the list to a floating-point number using the `float()` type-casting function.  (You'll need a `for` loop to run through the list.)

__Implement the function `parse_line_csv` below__ to carry out these three steps, then run the cell below to run it on `sample_line` and check that you parsed it correctly.

In [2]:
sample_line = '1,3.31,-0.27,7.79\n'

def parse_line_csv(line):
    ### BEGIN SOLUTION
    strip_line = line.strip()
    line_list = strip_line.split(',')
    number_list = []
    for x in line_list:
        number_list.append(float(x))
        
    return number_list
    ### END SOLUTION
    

In [3]:
parsed_line = parse_line_csv(sample_line)
print(parsed_line)

import numpy.testing as npt
npt.assert_allclose(parsed_line, [1.0, 3.31, -0.27, 7.79])

[1.0, 3.31, -0.27, 7.79]


### Part B

Now you're ready to process the whole data file!  In the cell below, use the `with open(...) as ...` syntax to open the file `example_data_tut26.csv`, and then use `readline()` or `readlines()` and your function from part A to create a two-dimensional NumPy array containing the data.  __Save it to a variable called `proc_data`.__

_(Hint: don't forget the header line!  Since it contains metadata and not data, you should store that to a different variable or discard it entirely.)_

_(Another hint: it's easiest to make a list of lists, and then use `np.array()` to typecast at the end.  You could also allocate an array of zeroes and then fill it in, line by line, but that requires knowing the exact dimensions of the data before you start by looking at the file - not always practical!)_

In [4]:
data_filename = 'example_data_tut26.csv'

### BEGIN SOLUTION
with open(data_filename) as dfile:
    header = dfile.readline()
    raw_lines = dfile.readlines()

proc_data = []
for line in raw_lines:
    proc_data.append(parse_line_csv(line))
    
proc_data = np.array(proc_data)

print(header)
print(proc_data)
### END SOLUTION


trial,x,y,t

[[ 1.    3.31 -0.27  7.79]
 [ 2.    0.41 -0.91  4.22]
 [ 3.   -2.35 -0.54  9.24]
 [ 4.   -1.62 -1.52 10.35]
 [ 5.   -0.98 -4.51 15.99]
 [ 6.   -3.86  3.76  7.4 ]
 [ 7.    4.67 -3.84 10.65]
 [ 8.    4.36 -0.61 16.65]
 [ 9.   -1.29  2.53 13.74]
 [10.   -4.63  3.13  7.47]
 [11.    0.46  4.39  0.68]
 [12.   -0.43  2.67  2.86]]


In [5]:
print(proc_data)
print(proc_data.shape)

import numpy.testing as npt
npt.assert_allclose(proc_data[9], [10, -4.63, 3.13, 7.47])
assert proc_data.shape == (12,4)

[[ 1.    3.31 -0.27  7.79]
 [ 2.    0.41 -0.91  4.22]
 [ 3.   -2.35 -0.54  9.24]
 [ 4.   -1.62 -1.52 10.35]
 [ 5.   -0.98 -4.51 15.99]
 [ 6.   -3.86  3.76  7.4 ]
 [ 7.    4.67 -3.84 10.65]
 [ 8.    4.36 -0.61 16.65]
 [ 9.   -1.29  2.53 13.74]
 [10.   -4.63  3.13  7.47]
 [11.    0.46  4.39  0.68]
 [12.   -0.43  2.67  2.86]]
(12, 4)


### Part C

Now our data is ready to use!  Let's apply a transformation, say we want to convert the middle two columns (x,y) to a single distance $d = \sqrt{x^2 + y^2}$.  That's easy enough to do:

In [6]:
distance_data = np.zeros((12,2))
distance_data[:,0] = proc_data[:,0]
distance_data[:,1] = np.sqrt(proc_data[:,1]**2 + proc_data[:,2]**2)

print(distance_data)

[[ 1.          3.32099383]
 [ 2.          0.99809819]
 [ 3.          2.41124449]
 [ 4.          2.22144097]
 [ 5.          4.61524647]
 [ 6.          5.38861763]
 [ 7.          6.04603176]
 [ 8.          4.40246522]
 [ 9.          2.83989436]
 [10.          5.58872078]
 [11.          4.41403444]
 [12.          2.70440382]]


Now we'd like to __write the transformed data out__ to a new file called `distances.csv`.  Use the `with open(..., 'w') as ...` context syntax to open `distances.csv` for writing, and then use the `.write()` file method to write lines to the file.  

__Use string formatting with the `g` format code__ for both numbers, and don't forget to include the newline `\n` at the end of every line!

In [9]:

### BEGIN SOLUTION
with open('distances.csv', 'w') as dist_file:
    for row in distance_data:
        dist_file.write('%g,%g\n' % (row[0], row[1]))
### END SOLUTION


In [10]:
with open('distances.csv', 'r') as dist_file:
    dlines = dist_file.readlines()

print(dlines)
assert len(dlines) == 12
assert dlines[4] == '5,4.61525\n'

['1,3.32099\n', '2,0.998098\n', '3,2.41124\n', '4,2.22144\n', '5,4.61525\n', '6,5.38862\n', '7,6.04603\n', '8,4.40247\n', '9,2.83989\n', '10,5.58872\n', '11,4.41403\n', '12,2.7044\n']


## T26.2 - Data mining the weather

Let's try working with some real data!  The provided file `weather_data_boulder_0918.csv` contains [NOAA weather data](https://www.ncdc.noaa.gov/cdo-web/) for various weather stations in the vicinity of Boulder, taken over 28 days in the month of September 2018.  The columns, in order, are:

* Weather station ID
* Weather station name
* Date of observation
* Precipitation total (inches)
* Average temperature (degrees F)
* Max temperature (degrees F)
* Min temperature (degrees F)

Since this is a real dataset, we'll encounter many real-world data wrangling problems: "cleaning" out extraneous data we don't care about, transforming and combining data, and so on.

Once again, start by __opening up the raw data file and looking through it__ to get a sense for the raw data.  One line will look something like this:

"USS0005J42S","NIWOT, CO US","2018-09-14","0.00","57","72","41"

In fact, this line with all data filled in is _rare_: most of the stations reported seem to only measure precipitation, so their temperature columns are missing entirely.  Of the stations reporting temperatures, most only record max/min and not the average.  Real data is often messy!


### Part A

Notice that this file is especially challenging to parse: this is a variation on CSV where the data are contained in double quotes `""` and _then_ separated by commas.  This is done so that commas can be used _inside_ the dataset, as in the station name above.

As a warm-up, I've included a single line of the data file as a string below.  __Extract the precipitation, max temperature, and min temperature__ from this string as a 3-entry NumPy array.


_(Hint: if you just pretend this is a regular CSV file and split on the commas, you'll end up breaking apart the station name field.  But if you only want the precipitation and temperature data, you can just ignore the name - but keep track of which column ends up where in the list after using `split`...)_

_(Another hint: the double quotes `"` will only get in your way here!  You can remove all the instances of a character from a string by using the `.replace()` method.  For example, `"hello world".replace('l', '')` will give you the string `'heo word'`.)_

In [11]:
wline = '"USS0005J42S","NIWOT, CO US","2018-09-14","0.00","57","72","41"\n'


### BEGIN SOLUTION
wline.strip().replace('"', '').split(',')
### END SOLUTION


['USS0005J42S', 'NIWOT', ' CO US', '2018-09-14', '0.00', '57', '72', '41']

### Part B

Now parse the whole data file, and __create a 2-D NumPy array containing the precipitation, max temperature, and min temperature__ for __only a single Boulder station, ID `USW00094075`.__  There are two ways you can do this:

1. Pretend this is a regular CSV file, and parse it by splitting on the commas as you did for the single line in part A.
2. Use the `csv` module [see the documentation here](https://docs.python.org/3/library/csv.html), and use a `csv.reader` to parse the dataset.  The `csv` module can recognize variations of CSV like this one and will deal with the quotes properly if you set the `quotechar` argument.

In [12]:
weather_filename = 'weather_data_boulder_0918.csv'
boulder_data = []

### BEGIN SOLUTION

# First method:
with open(weather_filename, 'r') as weather_file:
    for line in weather_file:
        split_data = line.strip().replace('"', '').split(',')
        #print(split_data)
        if split_data[0] == 'USW00094075':
            boulder_data.append([split_data[4], split_data[6], split_data[7]])

            
boulder_data = np.array(boulder_data, np.float)

# Second method:
boulder_data = []
with open(weather_filename, 'r') as weather_file:
    reader = csv.reader(weather_file, delimiter=',', quotechar='"')
    for line in reader:
        if line[0] == 'USW00094075':
            boulder_data.append([line[3], line[5], line[6]])

boulder_data = np.array(boulder_data, np.float)
### END SOLUTION


print(boulder_data)
    
    

[[0.0e+00 6.4e+01 3.5e+01]
 [0.0e+00 6.0e+01 4.0e+01]
 [0.0e+00 6.3e+01 4.0e+01]
 [2.7e-01 6.0e+01 3.9e+01]
 [6.0e-02 5.6e+01 3.4e+01]
 [4.0e-02 6.0e+01 3.0e+01]
 [6.0e-02 6.8e+01 3.3e+01]
 [0.0e+00 6.7e+01 3.8e+01]
 [0.0e+00 7.1e+01 3.6e+01]
 [0.0e+00 7.1e+01 3.9e+01]
 [0.0e+00 7.2e+01 4.6e+01]
 [0.0e+00 7.3e+01 4.3e+01]
 [0.0e+00 7.1e+01 4.2e+01]
 [0.0e+00 7.2e+01 4.2e+01]
 [0.0e+00 7.2e+01 4.7e+01]
 [0.0e+00 7.2e+01 4.7e+01]
 [0.0e+00 7.3e+01 4.2e+01]
 [7.0e-02 6.9e+01 4.4e+01]
 [0.0e+00 6.0e+01 3.1e+01]
 [0.0e+00 6.0e+01 2.6e+01]
 [0.0e+00 6.5e+01 3.7e+01]
 [0.0e+00 6.6e+01 4.1e+01]
 [1.9e-01 5.7e+01 3.8e+01]
 [0.0e+00 5.2e+01 2.5e+01]
 [0.0e+00 5.6e+01 2.6e+01]
 [0.0e+00 6.1e+01 3.5e+01]
 [0.0e+00 5.9e+01 3.5e+01]
 [0.0e+00 6.7e+01 4.3e+01]]


### Part C

Now extract the following quantities from your cleaned array of data in the cell below:

* Total precipitation;
* The lowest minimum temperature and highest maximum temperature;
* The average temperatures (obtained by averaging min/max temperature) on every Tuesday.

_(Hint: for the last one, use `np.mean()` and slicing to produce a 1d array containing the average temperature, then one more slice to cut the list down to the four Tuesdays only.  The data runs from 9/2 to 9/29, so the first day is a Sunday and the last is a Saturday.)_

In [13]:

### BEGIN SOLUTION
total_precip = np.sum(boulder_data[:,0])
print(total_precip)
min_temp = np.min(boulder_data[:,2])
print(min_temp)
max_temp = np.max(boulder_data[:,1])
print(max_temp)
avg_temps = np.mean(boulder_data[:,1:], axis=1)
print(avg_temps)

# First day is Sunday, so the first Tuesday is index 2.
# Skip by 7 after that using slice notation.
# Or give the four Tuesday indices explicitly.
print(avg_temps[2::7])
print(avg_temps[[2,9,16,23]])
### END SOLUTION

0.69
25.0
73.0
[49.5 50.  51.5 49.5 45.  45.  50.5 52.5 53.5 55.  59.  58.  56.5 57.
 59.5 59.5 57.5 56.5 45.5 43.  51.  53.5 47.5 38.5 41.  48.  47.  55. ]
[51.5 55.  57.5 38.5]
[51.5 55.  57.5 38.5]


### Part D (optional challenge)

Can you go back and report the lowest minimum and highest maximum temperature across _all weather stations_ in the file?  (This is tricky because many of the stations don't report any temperatures at all!)

In [14]:

### BEGIN SOLUTION
temp_data = []
with open(weather_filename, 'r') as weather_file:
    weather_file.readline() # Discard header
    reader = csv.reader(weather_file, delimiter=',', quotechar='"')
    
    for line in reader:
        if line[5] != '' and line[6] != '':
            temp_data.append([line[5], line[6]])

temp_data = np.array(temp_data, np.float)
print(temp_data.shape)

print(np.min(temp_data[:,1]))
print(np.max(temp_data[:,0]))
### END SOLUTION


(244, 2)
25.0
96.0
